In [1]:
from API_call import *
from functions import get_gamePks,read_gamePks
from datetime import datetime as dt
import os,re,csv
from os import walk
import statsapi as mlb
import pandas as pd
from database import *

In [2]:
get_gamePks([str(x) for x in range(2008,2020)])   

gamePks=read_gamePks()

In [3]:
db = MyDatabase('sqlite',dbname='mlb.db')

Engine(sqlite://///Users/schlinkertc/code/MLB/mlb.db)


In [4]:
Base.metadata.create_all(db.db_engine)
Base.metadata.bind=db.db_engine

In [5]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=db.db_engine)
session = Session()

In [6]:
def insert(call,session):
    mappers = [
        Game,Play,
        Pitch,
        Action,Runner,Credit,
        Matchup,Venue,
        Team,GameTeamLink,TeamRecord,
        Player
    ]
    for mapper in mappers:
        table = mapper.__tablename__
        records = call.__dict__[table]
        if mapper == Runner:
            for record in records:
                if record['movement_start']==None:
                    record['movement_start']='Home'
        session.bulk_insert_mappings(mapper,records)
    for mapper in [PitchData,HitData]:
        cols = [x.name for x in mapper.__table__.c]
        records = call.__dict__['pitches']
        df = pd.DataFrame.from_records(records)[cols]
        df.columns = [col.replace('pitchData_','') for col in df.columns]
        df.columns = [col.replace('hitData_','') for col in df.columns]
        df.dropna(axis=0,thresh=5,inplace=True)
        session.bulk_insert_mappings(mapper,df.to_dict(orient='records'))
    session.commit()
        
        

In [7]:
test_games = gamePks['2019'][600:650]

calls = [ API_call(g) for g in test_games ]

for call in calls:
    insert(call,session)

KeyError: "['pitchData_endSpeed', 'pitchData_coordinates_pZ', 'pitchData_typeConfidence', 'pitchData_coordinates_pX', 'pitchData_coordinates_x0', 'pitchData_plateTime', 'pitchData_coordinates_vX0', 'pitchData_coordinates_y0', 'pitchData_coordinates_aX', 'pitchData_coordinates_aZ', 'pitchData_breaks_spinRate', 'pitchData_breaks_spinDirection', 'pitchData_coordinates_aY', 'pitchData_breaks_breakY', 'pitchData_coordinates_pfxZ', 'pitchData_startSpeed', 'pitchData_coordinates_pfxX', 'pitchData_coordinates_vY0', 'pitchData_extension', 'pitchData_coordinates_z0', 'pitchData_zone', 'pitchData_coordinates_vZ0'] not in index"

In [16]:
dfs = {}
for table in db.db_engine.table_names():
    df = pd.read_sql(f"select * from {table}",db.db_engine)
    dfs[table]=df


In [32]:
for k in dfs.keys():
    df = dfs[k]
    for col in df.columns:
        if df[col].isna().sum()==df.shape[0]:
            print(k,col)

actions injuryType
actions reviewDetails_isOverturned
actions reviewDetails_reviewType
actions reviewDetails_challengeTeamId
games calenderEventId
players bateSide_description
players lastPlayedDate
players nameTitle
players deathDate
players deathCity
players deathStateProvince
players deathCountry
venues location_country
venues fieldInfo_left
venues fieldInfo_right


In [ ]:
dfs = {}
for k in calls[0].__dict__.keys():
    if k[0] != '_':
        records = [x for call in calls for x in call.__dict__[k]]
        df = pd.DataFrame.from_records(records)
        dfs[k] = df

In [ ]:
pitches = pitches[cols].dropna(axis=0,thresh=5)

In [ ]:
pitches.index.name = 'INDEX'

In [ ]:
dfs.keys()

In [ ]:
runners = dfs['runners']

In [ ]:
runners.info()

In [ ]:
pks = [
    'atBatIndex','playEndTime','gamePk',
    'details_playIndex','details_runner_id','details_eventType','movement_start'
]

In [ ]:
runners['movement_start'].fillna('home').value_counts().sum()

In [ ]:
runners[pks][runners[pks].duplicated()==True]

In [ ]:
runners[(runners['gamePk']==567281)&(runners['atBatIndex']==57)]

In [ ]:
runners[['atBatIndex','playEndTime','gamePk','details_playIndex','details_runner_id','movement_end']].duplicated()

In [ ]:
tr = dfs['team_records']

In [ ]:
tr.info()

In [ ]:
tr.head()